In [1]:
import pandas as pd
from io import StringIO
from owlready2 import *
from rdflib import RDFS, OWL, RDF
import types
import yaml

In [2]:
my_ontology = get_ontology("http://test.org/my_ontology")

In [3]:
class_pd = pd.read_csv(StringIO('''
class1,superclass
class1,Thing
class12,class1
class123,class12
class124,class12
class125,class12
'''))

In [4]:
# Create some subclasses as a tree

with my_ontology:
    for i, (clz,sup) in class_pd.iterrows():
        if sup == 'Thing':
            sup = Thing
        else:
            sup = my_ontology[sup]
        NewClass = types.new_class(clz, (sup,))

In [5]:
list(my_ontology.classes())

[my_ontology.class1,
 my_ontology.class12,
 my_ontology.class123,
 my_ontology.class124,
 my_ontology.class125]

In [6]:
# Create some instances

foods = {}

for food in ['bread', 'cheese', 'lettuce', 'crackers']:
    foods[food] = my_ontology['class123']()

In [7]:
class_pd

,class1,superclass
0,class1,Thing
1,class12,class1
2,class123,class12
3,class124,class12
4,class125,class12


In [8]:
property_pd = pd.read_csv(StringIO('''
property,superproperty
prop1,ObjectProperty
prop12,prop1
prop123,prop12
prop124,prop12
prop125,prop12
'''))

with my_ontology:
    Step = types.new_class('Step', (Thing,))

with my_ontology:
    class equipment(ObjectProperty):
        pass
    class step_equipment(equipment):
        domain           = [Step]

In [9]:
list(my_ontology.properties())

[my_ontology.equipment, my_ontology.step_equipment]

In [10]:
from rdflib import OWL
OWL.ObjectProperty

rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')

In [11]:
graph = default_world.as_rdflib_graph()

subclass_df = pd.DataFrame(
    graph.query('''
    SELECT ?c ?s WHERE {
        {
            ?c rdfs:subClassOf ?s .
            FILTER NOT EXISTS { ?c rdfs:label ?cl }
            FILTER NOT EXISTS { ?s rdfs:label ?sl }    
        } UNION {
            ?c rdfs:subClassOf ?s_ .
            FILTER NOT EXISTS { ?c rdfs:label ?cl }
            ?s_ rdfs:label ?s .
        } UNION {
            ?c_ rdfs:subClassOf ?s .
            FILTER NOT EXISTS { ?s rdfs:label ?sl }
            ?c_ rdfs:label ?c .
        } UNION {
            ?c_ rdfs:subClassOf ?s_ .
            ?c_ rdfs:label ?c .
            ?s_ rdfs:label ?s .
        }
    }
    ''',
            initNs={ 'rdfs': RDFS }
        ),
        columns=['source', 'target']
)
subclass_df['weight'] = 1

subclass_df

,source,target,weight
0,http://test.org/my_ontology#class1,http://www.w3.org/2002/07/owl#Thing,1
1,http://test.org/my_ontology#class12,http://test.org/my_ontology#class1,1
2,http://test.org/my_ontology#class123,http://test.org/my_ontology#class12,1
3,http://test.org/my_ontology#class124,http://test.org/my_ontology#class12,1
4,http://test.org/my_ontology#class125,http://test.org/my_ontology#class12,1
5,http://test.org/my_ontology#Step,http://www.w3.org/2002/07/owl#Thing,1
